In [1]:
import tensorflow as tf
import numpy as np
from collections import Counter

In [177]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [44]:
#Example 1
embedding_ph = tf.placeholder(tf.float32, [5, 10]) 
embeddings = tf.Variable(tf.random_uniform([5, 10], -1.0, 1.0), dtype=tf.float32, name="embeddings")
embeddings_init = embeddings.assign(embedding_ph)
shape = tf.reduce_sum(embeddings_init)
embedding_weights = np.ones((5, 10))
sess.run([shape], feed_dict={embedding_ph: embedding_weights})

[50.0]

In [6]:
words = ["a", "b", "a", "a", "c", "c"]
counter = Counter(words)
counter

Counter({'a': 3, 'b': 1, 'c': 2})

In [13]:
remove_list = [w for w, f in counter.items() if f < 2]
list(map(counter.pop, remove_list))


Counter({'a': 3, 'c': 2})

In [21]:
education = tf.contrib.layers.sparse_column_with_keys(
  column_name="gender", keys=["Master", "Bachelor"])
gender = tf.contrib.layers.sparse_column_with_keys(
  column_name="gender", keys=["Female", "Male"])
education_x_occupation = tf.contrib.layers.crossed_column([gender, education], hash_bucket_size=int(1e4))


In [22]:
education_x_occupation

_CrossedColumn(columns=(_SparseColumnKeys(column_name='gender', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Female', 'Male'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.string), _SparseColumnKeys(column_name='gender', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Master', 'Bachelor'), num_oov_buckets=0, vocab_size=2, default_value=-1), combiner='sum', dtype=tf.string)), hash_bucket_size=10000, hash_key=None, combiner='sum', ckpt_to_load_from=None, tensor_name_in_ckpt=None)

In [27]:
import tempfile
import urllib
test_file = tempfile.NamedTemporaryFile()
urllib.request.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

import pandas as pd
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]
df = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation",
                       "relationship", "race", "gender", "native_country"]

categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      dense_shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}

In [47]:
sess = tf.InteractiveSession()
result = sess.run(categorical_cols)
#categorical_cols['education'].eval()


In [53]:
type(categorical_cols)

dict

## Getting Started with Tensorflow
https://www.tensorflow.org/get_started/get_started

In [5]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [62]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [66]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
adder_node

<tf.Tensor 'add_1:0' shape=<unknown> dtype=float32>

In [68]:
sess.run(adder_node, {a: [5], b:[3, 1]})

array([ 8.,  6.], dtype=float32)

In [97]:
W = tf.Variable([.3], tf.float32, name='Weight')
b = tf.Variable([-.3], tf.float32, name="Bias")
x = tf.placeholder(tf.float32)
linear_model = W * x + b

init = tf.global_variables_initializer()
sess.run(init)
sess.run(linear_model, {x: 2})

array([ 0.30000001], dtype=float32)

In [102]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run([squared_deltas, loss], {x:[1,2,3,4], y:[0,-1,-2,-3]}))




[array([  0.        ,   1.68999982,   6.75999928,  15.21000099], dtype=float32), 23.66]


In [106]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

In [108]:
print(sess.run([W, b]))


[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [111]:
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
features

[_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)]

In [3]:
import numpy as np
import tensorflow as tf
# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)
# define our data set
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=10)

# train
estimator.fit(input_fn=input_fn, steps=5)
# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=1))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j1/vq1fgd095n75n3lcjx25nv1r0000gn/T/tmplmznnfvo', '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_evaluation_master': '', '_num_worker_replicas': 0, '_environment': 'local', '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a28d7f0>, '_is_chief': True, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_task_id': 0, '_task_type': None, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/j1/vq1fgd095n75n3lcjx25nv1r0000gn/T/tmplmznnfvo/model.ckpt.
INFO:tensorflow:step = 1, loss = 7.51949535745
INFO:tensorflow:Saving checkpoints for 5 into /var/folders/j1/vq1fgd095n75n3lcjx25nv1r0000gn/T/tmplmzn

In [4]:
sess = tf.InteractiveSession()
a = input_fn()[0]
list(sess.run(a['x']))



NameError: name 'input_fn' is not defined

### New functionalities

- https://www.tensorflow.org/get_started/get_started
    - tf.group(optimizer.minimize(loss),tf.assign_add(global_step, 1))
    - tf.contrib.learn.ModelFnOps
    - tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)
- https://www.tensorflow.org/get_started/mnist/mechanics
    - with tf.Graph().as_default(): (command that indicates all of the built ops are to be associated with the default global tf.Graph instance.)
    - tf.nn.in_top_k(logits, labels, 1)
    - tf.to_int64(labels)
- Other useful functions
    - tf.reset_default_graph()
    - [n.name for n in tf.get_default_graph().as_graph_def().node]  # Print nodes in the default graph.
    - tf.get_default_session()
    - run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)  # Execution stats
    - tf.gfile.Exists(FLAGS.log_dir)
    - tf.gfile.DeleteRecursively(FLAGS.log_dir)
    - tf.gfile.MakeDirs(FLAGS.log_dir)
    - tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TRAINING, target_dtype=np.int, features_dtype=np.float32)
    - tf.contrib.layers.real_valued_column("", dimension=4)
    - tf.contrib.layers.crossed_column([age_buckets, education, occupation], hash_bucket_size=int(1e6))
    - with tf.device("/cpu:0"):  # Pin a variable to CPU.
    - w2 = tf.Variable(weights.initialized_value(), name="w2")  # `tf.global_variables_initializer()` initializes variables in parallel, so more attention is required if one variable uses other variable's value. 
    - tf.variable_scope("foo", initializer=tf.constant_initializer(0.4))  # default initializer
    - tf.train.match_filenames_once
    - 



In [161]:
with tf.variable_scope("foo", reuse=True):
    a = tf.get_variable("v", [1])
sess.run(tf.global_variables_initializer())

In [194]:

tf.reset_default_graph()

In [195]:

with tf.variable_scope("bar"):
    b = tf.get_variable("b", [5])
    with tf.variable_scope("baz") as other_scope:
        b = tf.get_variable("b", [5])

In [196]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar", reuse=True):
    b = tf.get_variable("b", [5])
    with tf.variable_scope("baz") as other_scope:
        #assert other_scope.reuse == False, "reuse not false"
        b = tf.get_variable("b", [5])
#sess.run(tf.global_variables_initializer())

In [24]:
dir(tf.get_default_graph())


['_ControlDependenciesController',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_function',
 '_add_op',
 '_apply_device_functions',
 '_as_graph_def',
 '_as_graph_element_locked',
 '_attr_scope',
 '_attr_scope_map',
 '_building_function',
 '_c_graph',
 '_check_not_finalized',
 '_collections',
 '_colocation_stack',
 '_container',
 '_control_dependencies_for_inputs',
 '_control_dependencies_stack',
 '_control_flow_context',
 '_current_control_dependencies',
 '_default_original_op',
 '_device_function_stack',
 '_finalized',
 '_functions',
 '_get_control_flow_context',
 '_get_function',
 '_gradient_override_map',
 '_graph_def_versions',
 '_handle_deleters',
 '_handle_feeders',
 '_handle_

In [50]:
g = tf.get_default_graph()

print(g.get_all_collection_keys())
print(g.get_collection("trainable_variables"))

['trainable_variables', 'variables']
[<tf.Variable 'embeddings:0' shape=(5, 10) dtype=float32_ref>]


In [2]:
tf.parse_example?